In [1]:
import numpy as np
import os
import scipy.io
import antropy

In [2]:
def loadData(folderName):
    data = []
    labels = []
    allFiles = os.listdir(folderName)
    for file in allFiles:
        loadedFile = scipy.io.loadmat(folderName+'/'+file)
        data.append(loadedFile['data'])
        labels.append(loadedFile['labels'])

    return np.array(data), np.array(labels)

In [3]:
data, labels = loadData("../Datasets/data_preprocessed_matlab")

In [4]:
# subject * video * channel * data
print(data.shape)
# subject * video * labels
print(labels.shape)

(32, 40, 40, 8064)
(32, 40, 4)


In [5]:
def getSpecificChannels():
    channels = np.array(['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz',
                        'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2'])
    selectedChannel = np.array(
        ['AF3', 'F3', 'F7', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F8', 'F4', 'AF4'])
    return np.where(np.in1d(channels, selectedChannel))[0]

In [6]:
channelIndices = getSpecificChannels()
channelIndices

array([ 1,  2,  3,  4,  7, 11, 13, 17, 19, 20, 21, 25, 29, 31])

In [7]:
# Select 14 channel
selectedChannelData = data[:,:,channelIndices,:]
print(selectedChannelData.shape)

(32, 40, 14, 8064)


In [8]:
# Remove first 20 second of video
sampleRate = 128
removed20SecData = selectedChannelData[:,:,:,sampleRate*20:sampleRate*60]

In [9]:
removed20SecData.shape

(32, 40, 14, 5120)

In [10]:
processedData = np.zeros((32,160,14,1280))
parts = [removed20SecData[:,:,:,:sampleRate*10],
         removed20SecData[:,:,:,sampleRate*10:sampleRate*20],
         removed20SecData[:,:,:,sampleRate*20:sampleRate*30],
         removed20SecData[:,:,:,sampleRate*30:]
        ]

for s in range(32):
    for v in range(40):
        for i in range(4):
            processedData[s,(v * 4)+i, :,:] = parts[i][s,v,:,:]
            
processedLabels = np.repeat(labels,4, axis=1)

In [11]:
print(processedData.shape)
print(processedLabels.shape)

(32, 160, 14, 1280)
(32, 160, 4)


In [12]:
valenceLabels = ((processedLabels[:, :, 0] >= 5) * 1)
arousalLabels = ((processedLabels[:, :, 1] >= 5) * 1)
dominanceLabels = ((processedLabels[:, :, 2] >= 5) * 1)

In [13]:
subjectCount = processedData.shape[0]
videoCount = processedData.shape[1]
channelCount = processedData.shape[2]

In [32]:
item = processedData[3][1][2]
print(antropy.katz_fd(item))

4.096828535020412


In [33]:
antropy.petrosian_fd(item)

1.0239918724610544

In [34]:
antropy.higuchi_fd(item)

1.7452070313240973

In [15]:
def extractFD(data,subjectCount,videoCount,channelCount):
    
    subjectFeature=[]
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
        
            channelFeatures=[]
            for ch in range(channelCount):
                item = item = data[s][v][ch]
            
                katz_fd = antropy.katz_fd(item)
                petrosian_fd = antropy.petrosian_fd(item)
                higuchi_fd = antropy.higuchi_fd(item)
                
                features = np.array([katz_fd,petrosian_fd,higuchi_fd])
                channelFeatures.append(features)
                
            videoFeatures.append(channelFeatures)
            
        subjectFeature.append(videoFeatures)
        
    return np.array(subjectFeature)


In [16]:
fdFeatures = extractFD(processedData,subjectCount,videoCount,channelCount)
print(fdFeatures.shape)

(32, 160, 14, 3)


In [17]:
fdFeatures = fdFeatures.reshape(fdFeatures.shape[0],fdFeatures.shape[1],fdFeatures.shape[2]*fdFeatures.shape[3])
print(fdFeatures.shape)

(32, 160, 42)


In [19]:
import pickle
fdFeaturesFolder = 'fdFeatures.dat'
with open(fdFeaturesFolder,'wb') as f:
    pickle.dump(fdFeatures,f)

In [21]:
with open(fdFeaturesFolder,'rb') as f:
    fdFeatures = pickle.load(f)
print(fdFeatures.shape)

(32, 160, 42)
